In [120]:
import globus_sdk
# Use native app credentials if you have a client ID
CLIENT_ID = "04dc20ba-d8e4-4a9a-9cc9-3ed3f3d49447" 

# Create a UserApp instance
app = globus_sdk.UserApp(
    app_name="Notebook Main App",
    client_id=CLIENT_ID)

search_client = globus_sdk.SearchClient(app=app)
# transfer_client = globus_sdk.TransferClient(app=app)
# probably need to add https here

# Login if needed (Nick says probably not needed)
app.login()

In [121]:
# INDEX_ID = "6871e83e-866b-41bc-8430-e3cf83b43bdc" # Old XPCS index.
INDEX_ID = "4428cfe6-611b-48db-81b2-167a7d9710ea"

In [139]:
import os
from IPython.display import Image
from IPython.display import display
from gladier_xpcs.nb_utils.metadata import extract_metadata_structure
from gladier_xpcs.nb_utils.metadata import extract_entry_key_values
import h5py
import matplotlib.pyplot as plt

##helpful functions (in development)
def make_query(query, field_name='dc.identifiers.identifier', filter_value='*',  limit=10):
    
    query_data = {
        'q':query,
        'limit': limit,
        'advanced' : True,
        'filters' : [
            {
                "type": "like",
                "field_name": field_name,
                "value": filter_value
            }
        ]
    }
    print(query_data)
    response = search_client.post_search(
        index_id=INDEX_ID,
        data=query_data
        )
    print("Query results: " + str(response['total']))
    return response



def parse_result_files(q_entry, local_path=None):
    entry_files = q_entry[0]['entries'][0]['content']['files'] ## this is a list
    result_file = None
    log_file = None
    image_files = []
    globus_url = 'globus://74defd5b-5f61-42fc-bcc4-834c9f376a4f'
    for file in entry_files:
        if file['filename'].endswith('results.hdf'):
            result_file = file['url'] if local_path is None else file['url'].replace(globus_url, local_path)
        elif file['filename'].endswith('.log'):
            log_file = file['url'] if local_path is None else file['url'].replace(globus_url, local_path)
        elif file['filename'].endswith(('.png', '.jpg', '.jpeg')):
            image_files.append(file['url'] if local_path is None else file['url'].replace(globus_url, local_path))
    return result_file, log_file, image_files


def plot_hdf(file: str, x: str = '/xpcs/qmap/dynamic_v_list_dim0', y: str = '/xpcs/temporal_mean/scattering_1d'):
    with h5py.File(file, 'r') as f:
        x_set = f[x]
        y_set = f[y]
    # Plotting
    plt.figure(figsize=(8, 5))
    plt.plot(x_set, y_set , marker='o', linestyle='-', linewidth=2, markersize=4)
    plt.xlabel("q (1/Å)", fontsize=12)
    plt.ylabel("Intensity (a.u.)", fontsize=12)
    plt.title("1D Scattering Curve", fontsize=14)
    plt.grid(True, which='both', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
    return plt.gcf()


def plot_images(image_files): # weird call?
    # display(Image(entry_files[4]['url'].replace('globus://74defd5b-5f61-42fc-bcc4-834c9f376a4f', str(here))))
    pass


# def add_plot



In [140]:
## This is the wrong query, just for testing
query = "bacon"  # Match all entries (or a specific keyword)
make_query(query=query)

{'q': 'bacon', 'limit': 10, 'advanced': True, 'filters': [{'type': 'like', 'field_name': 'dc.identifiers.identifier', 'value': '*'}]}
Query results: 0


GlobusHTTPResponse({"total":0,"gmeta":[],"@datatype":"GSearchResult","@version":"2017-09-01","offset":0,"count":0,"has_next_page":false})

In [141]:
## Full Milliron query
query = "milliron"  # Match all entries (or a specific keyword)
milli_response = make_query(query = query)

{'q': 'milliron', 'limit': 10, 'advanced': True, 'filters': [{'type': 'like', 'field_name': 'dc.identifiers.identifier', 'value': '*'}]}
Query results: 21972


In [142]:
D100_set = make_query(query = "milliron",  filter_value='*D100*')

{'q': 'milliron', 'limit': 10, 'advanced': True, 'filters': [{'type': 'like', 'field_name': 'dc.identifiers.identifier', 'value': '*D100*'}]}
Query results: 12


In [144]:

local_path = '/eagle/projects/XPCS-DATA-DYS'
result_file, log_file, image_files = parse_result_files(D100_set["gmeta"], local_path)
print(result_file)
print(log_file)
print(image_files)

/eagle/projects/XPCS-DATA-DYS/XPCSDATA/Automate/2025-1/milliron202503/rigaku_qmap_Sq360_Dq36_Lin_0501/D0044_D100_a0050_f100000_t534C_r00003/D0044_D100_a0050_f100000_t534C_r00003_results.hdf
/eagle/projects/XPCS-DATA-DYS/XPCSDATA/Automate/2025-1/milliron202503/rigaku_qmap_Sq360_Dq36_Lin_0501/D0044_D100_a0050_f100000_t534C_r00003/boost_corr.log
['/eagle/projects/XPCS-DATA-DYS/XPCSDATA/Automate/2025-1/milliron202503/rigaku_qmap_Sq360_Dq36_Lin_0501/D0044_D100_a0050_f100000_t534C_r00003/resources/D0044_D100_a0050_f100000_t534C_r00003_results/total_intensity_vs_time.png', '/eagle/projects/XPCS-DATA-DYS/XPCSDATA/Automate/2025-1/milliron202503/rigaku_qmap_Sq360_Dq36_Lin_0501/D0044_D100_a0050_f100000_t534C_r00003/resources/D0044_D100_a0050_f100000_t534C_r00003_results/D0044_D100_a0050_f100000_t534C_r00003_results_g2_corr_000_008.png', '/eagle/projects/XPCS-DATA-DYS/XPCSDATA/Automate/2025-1/milliron202503/rigaku_qmap_Sq360_Dq36_Lin_0501/D0044_D100_a0050_f100000_t534C_r00003/resources/D0044_D100_

In [145]:
plot_hdf(result_file)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/eagle/projects/XPCS-DATA-DYS/XPCSDATA/Automate/2025-1/milliron202503/rigaku_qmap_Sq360_Dq36_Lin_0501/D0044_D100_a0050_f100000_t534C_r00003/D0044_D100_a0050_f100000_t534C_r00003_results.hdf', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [113]:
G10_set = make_query(query = "milliron",  filter_value='*G10*')

{'q': 'milliron', 'limit': 10, 'advanced': True, 'filters': [{'type': 'like', 'field_name': 'dc.identifiers.identifier', 'value': '*G10*'}]}
Query results: 0


In [114]:

os.chdir('/eagle/projects/XPCS-DATA-DYS')
here = os.getcwd()
# print("ALL FILES ON ENTRY")
entry_files = D100_set['gmeta'][0]['entries'][0]['content']['files'] ## this is a list
result_file, log_file, image_files = parse_result_files(entry_files)
print(result_file)
print(log_file)
print(image_files)


FileNotFoundError: [Errno 2] No such file or directory: '/eagle/projects/XPCS-DATA-DYS'

In [33]:
f = h5py.File('/lus/eagle/projects/XPCS-DATA-DYS/XPCSDATA/Automate/2025-1/milliron202503/rigaku_qmap_Sq360_Dq36_Lin_0501/volfrac315068_26nm-frac_a0118_f100000_r00001/volfrac315068_26nm-frac_a0118_f100000_r00001_results.hdf', 'r')

f.keys()
print(f.keys())
print(f['xpcs'].keys())
print(f['xpcs']['qmap'].keys())

import h5py
import matplotlib.pyplot as plt

q = f['xpcs']['qmap']['dynamic_v_list_dim0'][()]
intensity = f['xpcs']['temporal_mean']['scattering_1d'][()]
print(q.shape)
print(intensity.shape)

# # Plotting
# plt.figure(figsize=(8, 5))
# plt.plot(q, intensity, marker='o', linestyle='-', linewidth=2, markersize=4)
# plt.xlabel("q (1/Å)", fontsize=12)
# plt.ylabel("Intensity (a.u.)", fontsize=12)
# plt.title("1D Scattering Curve", fontsize=14)
# plt.grid(True, which='both', linestyle='--', alpha=0.6)
# plt.tight_layout()
# plt.show()


<KeysViewHDF5 ['entry', 'xpcs']>
<KeysViewHDF5 ['multitau', 'qmap', 'spatial_mean', 'temporal_mean']>
<KeysViewHDF5 ['beam_center_x', 'beam_center_y', 'detector_distance', 'dynamic_index_mapping', 'dynamic_num_pts', 'dynamic_roi_map', 'dynamic_v_list_dim0', 'dynamic_v_list_dim1', 'energy', 'map_names', 'map_units', 'mask', 'pixel_size', 'static_index_mapping', 'static_num_pts', 'static_roi_map', 'static_v_list_dim0', 'static_v_list_dim1']>
(36,)
(1, 360)


In [34]:
## Conversation with QZ. search inside an experiment
# Search for *G10* and *D100*
# plot q = f['xpcs']['qmap']['dynamic_v_list_dim0'][()] vs. intensity = f['xpcs']['temporal_mean']['scattering_1d'][()]

In [ ]:
extract_metadata_structure(milli_response)

# Get key-value pairs for the first entry
entry = milli_response["gmeta"][0]
values = extract_entry_key_values(entry)

# Print all DC metadata
print("Dublin Core Metadata:")
for key, value in values["dc"]:
    print(f"{key}: {value}")

# Print all project metadata
print("\nProject Metadata:")
for key, value in values["project_metadata"]:
    print(f"{key}: {value}")

# Print all file metadata
print("\nFile Metadata:")
for key, value in values["files"]:
    print(f"{key}: {value}")